In [1]:
# alpha vantage api key: LUNP8XN9APICB31P

In [82]:
# importing libraries for our analysis
import pandas as pd
import plotly.figure_factory as ff
import plotly.graph_objs as go

### 1. Extracting stock price data from alpha vantage API

In [5]:
import requests

# API endpoint URL
url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=RELIANCE.BSE&outputsize=full&apikey=LUNP8XN9APICB31P"

try:
    # Send GET request to the API endpoint
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Access the response content (JSON data)
        data = response.json()

        # Process the data as needed
        print(data)
    else:
        # Handle unsuccessful request
        print("Request failed with status code:", response.status_code)

except requests.RequestException as e:
    # Handle connection errors or other exceptions
    print("Error:", e)

{'Meta Data': {'1. Information': 'Daily Time Series with Splits and Dividend Events', '2. Symbol': 'RELIANCE.BSE', '3. Last Refreshed': '2023-06-23', '4. Output Size': 'Full size', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2023-06-23': {'1. open': '2530.0', '2. high': '2532.8', '3. low': '2511.1001', '4. close': '2515.25', '5. adjusted close': '2515.25', '6. volume': '289996', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2023-06-22': {'1. open': '2561.7', '2. high': '2572.8999', '3. low': '2532.25', '4. close': '2535.6001', '5. adjusted close': '2535.6001', '6. volume': '61620', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2023-06-21': {'1. open': '2556.3501', '2. high': '2569.1001', '3. low': '2547.95', '4. close': '2563.6001', '5. adjusted close': '2563.6001', '6. volume': '144710', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2023-06-20': {'1. open': '2550.5', '2. high': '2564.0', '3. low': '2535.0', '4. close

In [38]:
# time-series data  dict
stockdata_ts = data['Time Series (Daily)']

# create list of open, high, low, close and volume lists
open = []
high = []
low = []
close = []
volume = []

for value in stockdata_ts.values():
    open.append(value['1. open'])
    high.append(value['2. high'])
    low.append(value['3. low'])
    close.append(value['4. close'])
    volume.append(value['6. volume'])

In [39]:
# create the dataframe above dataset
RIL_df = pd.DataFrame(data={'open':open,
                            'high':high,
                            'low':low,
                            'close':close,
                            'volume':volume}, index=stockdata_ts.keys())

In [55]:
# Print the dataframe
RIL_df.head()

,open,high,low,close,volume
2023-06-23,2530.0,2532.8,2511.1001,2515.25,289996
2023-06-22,2561.7,2572.8999,2532.25,2535.6001,61620
2023-06-21,2556.3501,2569.1001,2547.95,2563.6001,144710
2023-06-20,2550.5,2564.0,2535.0,2546.75,171359
2023-06-19,2582.2,2584.0,2543.3999,2551.5,165566


In [42]:
# print the metadata of the dataframe
RIL_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4554 entries, 2023-06-23 to 2005-01-03
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   open    4554 non-null   object
 1   high    4554 non-null   object
 2   low     4554 non-null   object
 3   close   4554 non-null   object
 4   volume  4554 non-null   object
dtypes: object(5)
memory usage: 213.5+ KB


In [61]:
# change the datatype of the dataframe
RIL_df['open'] = RIL_df['open'].astype('float')
RIL_df['high'] = RIL_df['high'].astype('float')
RIL_df['low'] = RIL_df['low'].astype('float')
RIL_df['close'] = RIL_df['close'].astype('float')
RIL_df['volume'] = RIL_df['volume'].astype('float')

In [62]:
RIL_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4554 entries, 2023-06-23 to 2005-01-03
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    4554 non-null   float64
 1   high    4554 non-null   float64
 2   low     4554 non-null   float64
 3   close   4554 non-null   float64
 4   volume  4554 non-null   float64
dtypes: float64(5)
memory usage: 213.5+ KB


In [76]:
# change the datetime index datatype to datetime
RIL_df.index = pd.to_datetime(RIL_df.index, format='%Y/%m/%d')

### 2. Data processing to find average trading volume dring certain time period

In [72]:
# define a function to fine average trading volume for a given number of days
def avg_volume(data, num_days):
    if num_days == 0:
        avg = round(data['volume'].mean(), 3)
    else:
        avg = round(data[:num_days]['volume'].mean(), 3)
    return avg

In [73]:
# find the last 365 days average trading volume
avg_volume(RIL_df, 365)

325861.299

### 3. Candlestick chart of the RIL stock price

In [83]:
# using plotly graph_objs let's plot candlestick charts of stocks
def candlestick_chart(df, num_days):
    trace = go.Candlestick(x=df.index[:num_days],
                          open=df['open'][:num_days],
                          high=df['high'][:num_days],
                          low=df['low'][:num_days],
                          close=df['close'][:num_days])
    layout ={
    'title': f"RIL Historical Stock Price",
    'xaxis': {'title': 'Date'
             },
    'yaxis': {'title': 'Stock Price (USD$)'}
}
    
    data = [trace]
    
    fig = go.Figure(data=data, layout=layout)
    fig.update_layout(xaxis_rangeslider_visible=False, paper_bgcolor='#4bd659')
    fig.show()

In [84]:
# plot the historical 
candlestick_chart(RIL_df, 365)